# [모듈 1.1] Inference NCF on INF2

# 1. 환경 셋업

## 1.1. 기본 세팅
사용하는 패키지는 import 시점에 다시 재로딩 합니다.

In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('./src')

전 노트북에서 훈련 후의 아티펙트를 가져옵니다.

In [2]:
# ! pip install pandas==1.2.0
# ! pip install numpy==1.20.0


In [3]:
# ! pip list | grep pandas
# ! pip list | grep numpy

In [4]:
import torch
import torch_neuronx


## 1.2. 배포 준비

### 이전 노트북에서 훈련된 모델의 S3 경로 확인

In [5]:
artifact_path = 'models/NeuMF-end.pth'

In [6]:
print("model artifact is assigend from : ", artifact_path)

model artifact is assigend from :  models/NeuMF-end.pth


### 추론을 위한  데이터 세트 로딩
- 전부 데이터를 로딩할 필요가 없지만, 여기서는 기존에 사용한 함수를 이용하기 위해서 전체 데이터를 로드 합니다. 


In [7]:
import data_utils 
# train_data, test_data, user_num ,item_num, train_mat = data_utils.load_all(test_num=100)

### 파라미터 생성
- 모델 로딩시에 아라 파라미터 사용 

In [8]:
class Params:
    def __init__(self):
        # self.epochs = 1        
        self.num_ng = 4
        self.batch_size = 256
        self.test_num_ng = 99
        self.factor_num = 32
        self.num_layers = 3
        self.dropout = 0.0
        # self.lr = 0.001
        self.top_k = 10
        self.out = True
        # self.gpu = "0"
                        
args = Params()
print("# of batch_size: ", args.batch_size)


# of batch_size:  256


# 2. 훈련된 모델 아티펙트 다운로드 및 압축해제
- 모델 아티펙트를 다운로드 합니다.
- 다운로드 받은 모델 아티펙트의 압축을 해제하고 모델 가중치인 models/model.pth 파일을 얻습니다.

# 3. 훈련된 모델 로딩


## 3.1. 모델 네트워크 설정 저장
- 모델 네트워크를 생성시에 사용할 설정값을 model_config.json 로 저장함.
- model_fn() 함수에서 모델 네트워크를 생성시에 사용 함.

In [9]:
import os
import config

model_data_dir = config.model_path
os.makedirs(model_data_dir, exist_ok=True)
print("model_data_dir: ", model_data_dir)

model_data_dir:  ./models/


In [10]:
user_num = 6040  
item_num = 3706
print("user_num: ", user_num, " item_num: ", item_num)

user_num:  6040  item_num:  3706


In [11]:
import json
from common_utils import save_json, load_json

model_config_dict = {
    'user_num': str(user_num),
    'item_num': str(item_num),
    'factor_num' : str(args.factor_num),
    'num_layers' : str(args.num_layers),
    'dropout' : str(args.dropout),
    'model_type': config.model
}

model_config_file = 'model_config.json'
model_config_file_path = os.path.join('src', model_config_file)

save_json(model_config_file_path, model_config_dict)
# model_config_dict = load_json(model_config_file_path)    
# model_config_dict

src/model_config.json is saved


'src/model_config.json'

## 3.2. 모델 로딩
- 복수개의 모델로 진행하기 위해서, 편의상 동일한 모델에서 생성 함.


In [12]:
from inference import model_fn

ncf_model = model_fn(config.model_path)


######## Staring model_fn() ###############
device:  cpu


# 4. Trition 서빙 준비

## 4.1. 샘플 입력 생성

In [13]:
import numpy as np
import torch

def create_dummy_input(batch_size):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print("Using {} device".format(device))

    user_np = np.zeros((1,100)).astype(np.int32)
    item_np = np.random.randint(low=1, high=1000, size=(1,100)).astype(np.int32)


    return (
        torch.repeat_interleave(torch.from_numpy(user_np), batch_size, 0),
        torch.repeat_interleave(torch.from_numpy(item_np), batch_size, 0),
    )



dummy_inputs = create_dummy_input(batch_size=1)

print("type: ", type(dummy_inputs))
print("len: ", len(dummy_inputs))


Using cpu device
type:  <class 'tuple'>
len:  2


## 4.2. 샘플 입력으로 모델 추론 테스트

In [14]:
print("tes")

tes


## 4.3. Torch Script 으로 변환

In [15]:
def convert_torch_script(model, dummy_inputs):
    # Compile the model for Neuron
    model_neuron = torch_neuronx.trace(model, dummy_inputs)
    
    return model_neuron

model_neuron = convert_torch_script(ncf_model, dummy_inputs)

#### test 1 #####
#### prediction size: 
 torch.Size([1, 100, 1])
#### prediction: 
 tensor([[[-1.9414],
         [-1.8735],
         [ 1.4246],
         [-0.7328],
         [-2.1308],
         [-0.7274],
         [ 1.4956],
         [-3.1558],
         [-0.6960],
         [-0.8369],
         [-2.6756],
         [-2.5667],
         [-1.5944],
         [ 1.8739],
         [-3.2315],
         [-2.7633],
         [-1.8741],
         [-0.8376],
         [-1.0394],
         [-1.6824],
         [-1.3185],
         [-0.3596],
         [-2.1842],
         [-3.5964],
         [ 0.8121],
         [ 0.9245],
         [-4.9187],
         [-3.6179],
         [ 2.5198],
         [-0.6039],
         [-0.4386],
         [-1.0476],
         [-1.7840],
         [-1.1848],
         [-1.5194],
         [-3.2381],
         [-4.3907],
         [-4.3857],
         [-3.8251],
         [-1.7962],
         [-2.0258],
         [-3.0666],
         [-4.5885],
         [-2.0000],
         [-0.6448],
         [-2.67

03/07/2023 02:09:06 PM WARNING 100723 [py.warnings]: /home/ubuntu/aws_neuron_venv_pytorch/bin/neuronx-cc:8: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  sys.exit(main())

03/07/2023 02:09:08 PM WARNING 100723 [WalrusDriver]: 0% PSUM demand before spilling
03/07/2023 02:09:08 PM WARNING 100723 [WalrusDriver]: spilling from PSUM cost about 0 cycles
03/07/2023 02:09:08 PM WARNING 100723 [WalrusDriver]: 0% PSUM utilization after allocation
03/07/2023 02:09:08 PM WARNING 100723 [WalrusDriver]: spilling from SB cost about 0 cycles
03/07/2023 02:09:08 PM WARNING 100723 [WalrusDriver]: 0 bytes/partition (0%) successfully pinned
03/07/2023 02:09:08 PM WARNING 100723 [WalrusD

In [16]:
model_neuron(dummy_inputs[0],dummy_inputs[1])

tensor([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0, 125, 834,  39, 770, 899, 464, 531, 791, 768, 790, 675, 727,
         844,  14, 855, 230, 698, 732, 884, 605, 686, 756,  99, 296, 213,  29,
         813, 497, 171, 129, 350, 625, 399, 249, 982, 197, 980, 962, 441, 455,
         761, 990, 367, 819, 236, 944, 851, 860,  43, 250, 560, 979, 435, 152,
          40, 420, 659, 348, 363, 679, 512, 447, 749, 841, 631, 531, 290, 913,
         696, 889,  51, 250, 685, 264, 753, 511, 888